In [ ]:
%%capture
%matplotlib widget

import matplotlib
import markdown 
from ipywidgets import widgets, HTML

from velocity_widget import ITSLIVE

matplotlib.rcParams['figure.figsize'] = [6, 4]
matplotlib.rcParams["figure.autolayout"] = True

velocity_widget = ITSLIVE()

In [ ]:
html_title = markdown.markdown("""

<div>
    <h2><center><a href="https://its-live.jpl.nasa.gov/"><img align="middle" src="https://its-live-data.s3.amazonaws.com/documentation/ITS_LIVE_logo.png" height="50"/></a></center></h2>
    <h3><center>Global Glacier Velocity Point Data Access</center></h3>
</div>

***

<b>Authors</b>: Mark Fahnestock, Alex S. Gardner, Luis Lopez, Ted Scambos, Maria Liukis, Chad A. Greene, Joseph H. Kennedy


""")


html_instructions = markdown.markdown(""" This widget allows you to select a set of point locations on a Leaflet-based global map, fetch all available ITS_LIVE glacier velocities for the locations, and plot the results.
Select the points by <b>double-clicking on the map</b>. You can also click and add a point using the "Add Point" button, the point location will be that of the last click on the map. - click and drag on the map to pan the field of view. Once points are chosen, hit "Make Plot" (bottom right corner) to produce a plot. You can drag individual points after they are placed to relocate them, and then "Make Plot" again. Click "Clear Points" to start over.
Underling data is stored on AWS S3 as Zarr datacubes and is accessed without an intermediate server. Glacier velocities in the "All Satellite" datacube collection are derived from all available Landsat 8, Sentinel-1A/B, Sentinel-2A/B imagery, while they are limited to Landsat 8 for now in the global dataset.

We can also watch a tutorial on Youtube by Alex Gardner:

<center><a href="https://www.youtube.com/watch?v=G7E7rE5npvg" target="_blank"><img width="50%" src="https://raw.githubusercontent.com/nasa-jpl/itslive-web/main/static/images/youtube-screen.jpg"></a></center>

Please refer to the <b>[project website](https://its-live.jpl.nasa.gov/)</b> for further product infomation and for appraopriate data citation.
""")


title = HTML(html_title, layout=widgets.Layout(width="100%",
                                               display='flex',
                                               align_items='stretch'))
instructions = widgets.Accordion(children=[HTML(html_instructions)], selected_index=None)
instructions.set_title(0,title="Instructions")

In [ ]:

dates_range = widgets.SelectionRangeSlider(
    options=[i for i in range(120)],
    index=(1, 90),
    continuous_update=False,
    description='Separation Range (days): ',
    orientation='horizontal',
    layout={'width': '90%',
            'display': 'flex'},
    style={'description_width': 'initial'})

variables =  widgets.Dropdown(
    options=['v', 'v_error', 'vx', 'vy'],
    description='Variable: ',
    disabled=False,
    value='v',
    layout={'width': '20%',
            'display': 'flex'},
    style={'description_width': 'initial'})

plot_type =  widgets.Dropdown(
    options=['points', 'satellite'],
    description='Plot By: ',
    disabled=False,
    value='points',
    layout={'width': '20%',
            'display': 'flex'},
    style={'description_width': 'initial'})

plot_button =  widgets.Button(
    description='Plot',
    button_style='primary',
    icon='line-chart',
    style={'description_width': 'initial'})

clear_button =  widgets.Button(
    description='Clear markers',
    # button_style='warning',
    icon="trash",
    style={'description_width': 'initial'})

latitude = widgets.BoundedFloatText(
    value=0.0,
    min=-90.0,
    max=90.0,
    step=0.1,
    description='Lat: ',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '20%',
            'display': 'flex'},
)

longitude = widgets.BoundedFloatText(
    value=0.0,
    min=-180.0,
    max=180.0,
    step=0.1,
    description='Lon: ',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '20%',
            'display': 'flex'},
)

add_button =  widgets.Button(
    description='Add Point',
    # button_style='info',
    icon="map-marker",
    style={'description_width': 'initial'})

include_running_mean =  widgets.Checkbox(
            value=False,
            description="Include Running Mean",
            style={'description_width': 'initial'},
            disabled=False,
            indent=False,
            tooltip="Plot running mean through each time series",
            layout=widgets.Layout(width="25%"),
        )


# If this congiguration changes we need to rerun the cell.
config = { 
    "plot": "v", # or other ITS_LIVE variables: vx, vy ...
    "min_separation_days": 1,
    "max_separation_days": 90,
    "color_by": "points", # valid values: satellite, points
    "verbose": True, # print operations
    "runnig_mean": True,
    "coords": {
        "latitude": latitude,
        "longitude": longitude
    }
}


plot_button.on_click(velocity_widget.plot_time_series)
clear_button.on_click(velocity_widget.clear_points)

def update_variable(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["plot"] = variables.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_range(change):
        if change['type'] == 'change' and change['name'] == 'value':
            start, end = change['new']
            config["min_separation_days"] = start
            config["max_separation_days"] = end
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_plottype(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["color_by"] = plot_type.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_mean(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["running_mean"] = include_running_mean.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def add_point(event):
    coords = (latitude.value, longitude.value)
    velocity_widget.add_point(coords)


add_button.on_click(add_point)
dates_range.observe(update_range, 'value')
plot_type.observe(update_plottype, 'value')
variables.observe(update_variable, 'value')
include_running_mean.observe(update_mean, 'value')

layout = widgets.Layout(align_items='stretch',
                        display='flex',
                        flex_flow='row wrap',
                        border='none',
                        grid_template_columns="repeat(auto-fit, minmax(420px, 1fr))",
                        # grid_template_columns='48% 48%',
                        width='99%',
                        height='100%')

velocity_widget.set_config(config)

velocity_widget.fig.canvas.capture_scroll = True


widgets.GridBox([widgets.VBox([title,
                 instructions], layout=widgets.Layout(width="100%")),
                 widgets.VBox([velocity_widget.map,
                               widgets.HBox([latitude, longitude, add_button, clear_button], layout=widgets.Layout(align_items="flex-start",
                                                                                                                   flex_flow='row wrap')),
                               dates_range,
                               widgets.HBox([plot_button, variables, plot_type, include_running_mean], layout=widgets.Layout(justify_content="flex-start",
                                                                                                                             flex_flow='row wrap'))],
                              layout=widgets.Layout(min_width="420px",
                                                    # display="flex",
                                                    # height="100%",
                                                    # max_height="100%",
                                                    max_width="100%")),
                  widgets.VBox([
                               velocity_widget.fig.canvas,
                               ], layout=widgets.Layout(min_width="420px",
                                                        overflow='scroll',
                                                        max_width="100%",
                                                        display='flex'))],
                 layout=layout)